<a href="https://colab.research.google.com/github/yash2907183/sentiment-analysis/blob/main/sentiment_analysis_of_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

**Importing the dependencies**

In [2]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data Collection - Kaggle API

In [3]:
kaggle_dictionary = json.load(open("kaggle (1).json"))

In [4]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [5]:
#setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"]=kaggle_dictionary['username']
os.environ["KAGGLE_KEY"]=kaggle_dictionary['key']

In [6]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other


In [7]:
!ls

 imdb-dataset-of-50k-movie-reviews.zip	'kaggle (1).json'   sample_data


In [8]:
#unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [9]:
!ls

'IMDB Dataset.csv'			'kaggle (1).json'
 imdb-dataset-of-50k-movie-reviews.zip	 sample_data


Loading the dataset

In [10]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [11]:
data.shape

(50000, 2)

In [12]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [13]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [14]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [15]:
data.replace({"sentiment":{"positive":1, "negative":0}}, inplace= True)

<ipython-input-15-511356d07890>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment":{"positive":1, "negative":0}}, inplace= True)


In [16]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [17]:
#split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state = 42)

In [18]:
print(train_data.shape)

(40000, 2)


In [19]:
print(test_data.shape)

(10000, 2)


Data preprocessing

In [20]:
#Tokenize the text data
tokenizer = Tokenizer(num_words=5000) #take the most common 5000 words from this data
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)
# pad_sequences ensures that all the data in train_data["review"] has the same length
# this is because the input to any neural netwwork should have the same uniform input shape/length

In [21]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [22]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [23]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [24]:
Y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


**LSTM - Long Short Term Memory**

In [35]:
# Build the model

model = Sequential()
model.add(Embedding(input_dim = 5000, output_dim = 128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2)) #20% of the data is going to flow as 0's, its to prevent overfitting, recurrent (RNN), helps in generalising
model.add(Dense(1, activation ="sigmoid")) #all the neurons are connected to single neuron, binary class. therefore, we should use sigmoid activation

In [36]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [37]:
model.build(input_shape=(None, 200))

In [39]:
model.summary() #parameters are calculated by multiplying input dimension with output dimension

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [40]:
#compile the model
model.compile(optimizer="adam", loss = "binary_crossentropy", metrics=["accuracy"])

**Training the Model**

In [41]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 193s 375ms/step - accuracy: 0.7298 - loss: 0.5221 - val_accuracy: 0.8390 - val_loss: 0.3713
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 204s 385ms/step - accuracy: 0.8589 - loss: 0.3473 - val_accuracy: 0.8429 - val_loss: 0.3633
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 383ms/step - accuracy: 0.8801 - loss: 0.3011 - val_accuracy: 0.7676 - val_loss: 0.4930
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 200s 381ms/step - accuracy: 0.7828 - loss: 0.4591 - val_accuracy: 0.8529 - val_loss: 0.3626
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 382ms/step - accuracy: 0.8902 - loss: 0.2784 - val_accuracy: 0.8655 - val_loss: 0.3403


In [42]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 35s 109ms/step - accuracy: 0.8706 - loss: 0.3256
Test Loss: 0.31684571504592896
Test Accuracy: 0.8756999969482422


**Build a Predictive System**

In [43]:
def predict_sentiment(review):
  #tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0]>0.5 else "negative"
  return sentiment

In [44]:
#example usage
review = "This movie was so bad. I hated every part of it"
sentiment = predict_sentiment(review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step
The sentiment of the review is: negative


In [45]:
#example usage
review2 = "This movie was so good purrr. I loved it"
sentiment = predict_sentiment(review2)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
The sentiment of the review is: positive
